In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 71.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 88.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import torchvision

from torchvision import transforms
import time
torch.manual_seed(17)

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
print(device)

cuda


In [4]:
transform = transforms.Compose([ 
    transforms.Resize((224, 224)), 
    transforms.ToTensor()
])

In [5]:
dataset = torchvision.datasets.ImageFolder('train', transform=transform)

In [6]:
len(dataset)

100000

In [7]:
#split the data
train_data, val_data, test_data = torch.utils.data.random_split(dataset, [80000, 10000, 10000])

In [8]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

In [30]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 200)
# Move models to device
teacher = teacher.to(device)

In [12]:

# model_path = 'best_teacher_model.pth'
# # Load the model weights
# teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 200)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:

# model_path = 'student_before_pruning.pth'
# # Load the model weights
# student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [14]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [15]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [16]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [17]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [18]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [20]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [31]:
# Fine-tune the teacher model
teacher = train_model(teacher, train_loader, val_loader, epochs=200, lr=0.001, patience=5)

0
Epoch 1/200 | Loss: 2.0593 | Val Accuracy: 74.06%
 New best model saved with validation accuracy: 74.06%
1
Epoch 2/200 | Loss: 0.8640 | Val Accuracy: 76.41%
 New best model saved with validation accuracy: 76.41%
2
Epoch 3/200 | Loss: 0.5448 | Val Accuracy: 76.62%
 New best model saved with validation accuracy: 76.62%
3
Epoch 4/200 | Loss: 0.3316 | Val Accuracy: 76.56%
 No improvement in validation accuracy (1/5)
4
Epoch 5/200 | Loss: 0.1921 | Val Accuracy: 77.56%
 New best model saved with validation accuracy: 77.56%
5
Epoch 6/200 | Loss: 0.1019 | Val Accuracy: 77.74%
 New best model saved with validation accuracy: 77.74%
6
Epoch 7/200 | Loss: 0.0574 | Val Accuracy: 77.66%
 No improvement in validation accuracy (1/5)
7
Epoch 8/200 | Loss: 0.0329 | Val Accuracy: 77.80%
 New best model saved with validation accuracy: 77.80%
8
Epoch 9/200 | Loss: 0.0221 | Val Accuracy: 77.92%
 New best model saved with validation accuracy: 77.92%
9
Epoch 10/200 | Loss: 0.0160 | Val Accuracy: 78.00%
 New

In [21]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [22]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [23]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [25]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [32]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 2.5908 | Train Acc: 95.53% | Val Acc: 56.69%
Epoch 2/50 | Train Loss: 1.2254 | Train Acc: 99.10% | Val Acc: 58.77%
Epoch 3/50 | Train Loss: 0.7198 | Train Acc: 99.83% | Val Acc: 60.37%
Epoch 4/50 | Train Loss: 0.5329 | Train Acc: 99.95% | Val Acc: 61.01%
Epoch 5/50 | Train Loss: 0.4462 | Train Acc: 99.98% | Val Acc: 61.32%
Epoch 6/50 | Train Loss: 0.4034 | Train Acc: 99.98% | Val Acc: 61.37%
Epoch 7/50 | Train Loss: 0.3741 | Train Acc: 99.98% | Val Acc: 61.65%
Epoch 8/50 | Train Loss: 0.3559 | Train Acc: 99.98% | Val Acc: 61.67%
Epoch 9/50 | Train Loss: 0.3395 | Train Acc: 99.98% | Val Acc: 61.68%
Epoch 10/50 | Train Loss: 0.3279 | Train Acc: 99.98% | Val Acc: 61.55%
Epoch 11/50 | Train Loss: 0.3166 | Train Acc: 99.98% | Val Acc: 61.65%
Epoch 12/50 | Train Loss: 0.3063 | Train Acc: 99.98% | Val Acc: 61.89%
Epoch 13/50 | Train Loss: 0.2991 | Train Acc: 99.98% | Val Acc: 61.59%
Epoch 14/50 | Train Loss: 0.2922 | Train Acc: 99.98% | Val Acc: 61.72%
Epoch 15/50 | T

In [33]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 78.46%
Student Model Test Accuracy Before Pruning: 62.19%


## 93% Sparsity

In [34]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [35]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [36]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 0.51%


In [37]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 29.42%
Epoch 1/200 | Train Loss: 4.3579 | Train Acc: 15.95%
Validation Loss: 3.2594 | Validation Acc: 29.42% | Sparsity: 93.11%

New best model saved with Val Accuracy: 41.56%
Epoch 2/200 | Train Loss: 2.5401 | Train Acc: 41.16%
Validation Loss: 2.4974 | Validation Acc: 41.56% | Sparsity: 93.11%

New best model saved with Val Accuracy: 46.10%
Epoch 3/200 | Train Loss: 1.9987 | Train Acc: 51.41%
Validation Loss: 2.2510 | Validation Acc: 46.10% | Sparsity: 93.11%

New best model saved with Val Accuracy: 48.25%
Epoch 4/200 | Train Loss: 1.7396 | Train Acc: 56.88%
Validation Loss: 2.1539 | Validation Acc: 48.25% | Sparsity: 93.11%

New best model saved with Val Accuracy: 49.58%
Epoch 5/200 | Train Loss: 1.5582 | Train Acc: 61.03%
Validation Loss: 2.0967 | Validation Acc: 49.58% | Sparsity: 93.11%

New best model saved with Val Accuracy: 50.25%
Epoch 6/200 | Train Loss: 1.4234 | Train Acc: 63.96%
Validation Loss: 2.0655 | Validation Acc: 50.25% | Spar

In [38]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 51.74%


In [39]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [40]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 46s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [41]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 13.4261 | Train Acc: 45.14% | Val Loss: 2.4397 | Val Acc: 48.49% | Sparsity: 93.11%
Epoch 2/50 | Train Loss: 9.2128 | Train Acc: 62.95% | Val Loss: 2.2383 | Val Acc: 52.33% | Sparsity: 93.11%
Epoch 3/50 | Train Loss: 7.7465 | Train Acc: 69.39% | Val Loss: 2.2246 | Val Acc: 53.10% | Sparsity: 93.11%
Epoch 4/50 | Train Loss: 6.7792 | Train Acc: 73.76% | Val Loss: 2.2012 | Val Acc: 54.59% | Sparsity: 93.11%
Epoch 5/50 | Train Loss: 6.0349 | Train Acc: 77.25% | Val Loss: 2.2135 | Val Acc: 54.97% | Sparsity: 93.11%
Epoch 6/50 | Train Loss: 5.4562 | Train Acc: 80.12% | Val Loss: 2.2291 | Val Acc: 54.40% | Sparsity: 93.11%
Epoch 7/50 | Train Loss: 4.9603 | Train Acc: 82.44% | Val Loss: 2.2161 | Val Acc: 53.69% | Sparsity: 93.11%
Epoch 8/50 | Train Loss: 4.5286 | Train Acc: 84.54% | Val Loss: 2.2590 | Val Acc: 53.57% | Sparsity: 93.11%
Epoch 9/50 | Train Loss: 4.1383 | Train Acc: 86.40% | Val Loss: 2.2403 | Val Acc: 53.78% | Sparsity: 93.11%
Epoch 10/50 | Train Loss: 3

In [42]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 53.79%


In [43]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [44]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 53s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [45]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 32.9822 | Train Acc: 47.87% | Val Loss: 2.3725 | Val Acc: 49.29% | Sparsity: 93.11%
Epoch 2/50 | Train Loss: 24.0918 | Train Acc: 62.03% | Val Loss: 2.2526 | Val Acc: 52.02% | Sparsity: 93.11%
Epoch 3/50 | Train Loss: 20.8340 | Train Acc: 67.53% | Val Loss: 2.1998 | Val Acc: 53.33% | Sparsity: 93.11%
Epoch 4/50 | Train Loss: 18.5712 | Train Acc: 71.50% | Val Loss: 2.2473 | Val Acc: 53.02% | Sparsity: 93.11%
Epoch 5/50 | Train Loss: 16.8159 | Train Acc: 74.71% | Val Loss: 2.2101 | Val Acc: 52.87% | Sparsity: 93.11%
Epoch 6/50 | Train Loss: 15.3038 | Train Acc: 77.57% | Val Loss: 2.2537 | Val Acc: 52.42% | Sparsity: 93.11%
Epoch 7/50 | Train Loss: 13.8867 | Train Acc: 80.14% | Val Loss: 2.2255 | Val Acc: 53.39% | Sparsity: 93.11%
Epoch 8/50 | Train Loss: 12.7689 | Train Acc: 82.13% | Val Loss: 2.2549 | Val Acc: 52.50% | Sparsity: 93.11%
Epoch 9/50 | Train Loss: 11.6378 | Train Acc: 84.35% | Val Loss: 2.2415 | Val Acc: 53.16% | Sparsity: 93.11%
Epoch 10/50 | Train

In [46]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 52.11%


# 98% of Sparsity

In [47]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [48]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 28s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [49]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 4.74%
Epoch 1/200 | Train Loss: 5.2237 | Train Acc: 2.33%
Validation Loss: 4.9673 | Validation Acc: 4.74% | Sparsity: 97.56%

New best model saved with Val Accuracy: 15.13%
Epoch 2/200 | Train Loss: 4.5510 | Train Acc: 9.47%
Validation Loss: 4.1147 | Validation Acc: 15.13% | Sparsity: 97.56%

New best model saved with Val Accuracy: 24.89%
Epoch 3/200 | Train Loss: 3.6774 | Train Acc: 20.93%
Validation Loss: 3.3649 | Validation Acc: 24.89% | Sparsity: 97.56%

New best model saved with Val Accuracy: 30.82%
Epoch 4/200 | Train Loss: 3.1443 | Train Acc: 28.74%
Validation Loss: 3.0213 | Validation Acc: 30.82% | Sparsity: 97.56%

New best model saved with Val Accuracy: 34.57%
Epoch 5/200 | Train Loss: 2.8503 | Train Acc: 34.04%
Validation Loss: 2.8266 | Validation Acc: 34.57% | Sparsity: 97.56%

New best model saved with Val Accuracy: 37.12%
Epoch 6/200 | Train Loss: 2.6522 | Train Acc: 37.63%
Validation Loss: 2.7025 | Validation Acc: 37.12% | Sparsity

In [50]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 47.71%


In [51]:

# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


# Retrained with KD

In [52]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [53]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 62.19%


In [54]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 30s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [55]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 46.5602 | Train Acc: 25.36% | Val Loss: 2.9118 | Val Acc: 36.93% | Sparsity: 97.56%
Epoch 2/50 | Train Loss: 36.0633 | Train Acc: 42.82% | Val Loss: 2.5652 | Val Acc: 43.14% | Sparsity: 97.56%
Epoch 3/50 | Train Loss: 32.7026 | Train Acc: 48.26% | Val Loss: 2.4702 | Val Acc: 45.11% | Sparsity: 97.56%
Epoch 4/50 | Train Loss: 30.6133 | Train Acc: 51.74% | Val Loss: 2.4115 | Val Acc: 46.74% | Sparsity: 97.56%
Epoch 5/50 | Train Loss: 29.1158 | Train Acc: 54.12% | Val Loss: 2.3706 | Val Acc: 47.74% | Sparsity: 97.56%
Epoch 6/50 | Train Loss: 27.9455 | Train Acc: 56.08% | Val Loss: 2.2967 | Val Acc: 49.34% | Sparsity: 97.56%
Epoch 7/50 | Train Loss: 26.9808 | Train Acc: 57.91% | Val Loss: 2.3064 | Val Acc: 49.82% | Sparsity: 97.56%
Epoch 8/50 | Train Loss: 26.1604 | Train Acc: 59.42% | Val Loss: 2.2794 | Val Acc: 49.66% | Sparsity: 97.56%
Epoch 9/50 | Train Loss: 25.4633 | Train Acc: 60.62% | Val Loss: 2.2977 | Val Acc: 49.05% | Sparsity: 97.56%
Epoch 10/50 | Train

In [56]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 50.07%


In [57]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [58]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 31s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [59]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 19.1698 | Train Acc: 18.37% | Val Loss: 3.3076 | Val Acc: 31.60% | Sparsity: 97.56%
Epoch 2/50 | Train Loss: 14.9123 | Train Acc: 37.73% | Val Loss: 2.8577 | Val Acc: 39.68% | Sparsity: 97.56%
Epoch 3/50 | Train Loss: 13.2350 | Train Acc: 45.12% | Val Loss: 2.6228 | Val Acc: 43.53% | Sparsity: 97.56%
Epoch 4/50 | Train Loss: 12.2541 | Train Acc: 49.32% | Val Loss: 2.5229 | Val Acc: 45.37% | Sparsity: 97.56%
Epoch 5/50 | Train Loss: 11.5527 | Train Acc: 52.38% | Val Loss: 2.4525 | Val Acc: 47.32% | Sparsity: 97.56%
Epoch 6/50 | Train Loss: 11.0221 | Train Acc: 54.87% | Val Loss: 2.4072 | Val Acc: 48.01% | Sparsity: 97.56%
Epoch 7/50 | Train Loss: 10.6098 | Train Acc: 56.55% | Val Loss: 2.4083 | Val Acc: 48.69% | Sparsity: 97.56%
Epoch 8/50 | Train Loss: 10.2647 | Train Acc: 58.23% | Val Loss: 2.3425 | Val Acc: 49.61% | Sparsity: 97.56%
Epoch 9/50 | Train Loss: 9.9430 | Train Acc: 59.73% | Val Loss: 2.3698 | Val Acc: 49.42% | Sparsity: 97.56%
Epoch 10/50 | Train 

In [60]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 50.64%


## 90% Sparsity

In [61]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [62]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 40s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [63]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 41.80%
Epoch 1/100 | Train Loss: 3.1422 | Train Acc: 35.62%
Validation Loss: 2.6129 | Validation Acc: 41.80% | Sparsity: 89.22%

New best model saved with Val Accuracy: 47.98%
Epoch 2/100 | Train Loss: 1.6920 | Train Acc: 59.09%
Validation Loss: 2.2863 | Validation Acc: 47.98% | Sparsity: 89.22%

New best model saved with Val Accuracy: 50.02%
Epoch 3/100 | Train Loss: 1.3231 | Train Acc: 66.88%
Validation Loss: 2.1767 | Validation Acc: 50.02% | Sparsity: 89.22%

New best model saved with Val Accuracy: 51.11%
Epoch 4/100 | Train Loss: 1.1100 | Train Acc: 71.51%
Validation Loss: 2.1598 | Validation Acc: 51.11% | Sparsity: 89.22%

New best model saved with Val Accuracy: 51.90%
Epoch 5/100 | Train Loss: 0.9548 | Train Acc: 75.21%
Validation Loss: 2.1443 | Validation Acc: 51.90% | Sparsity: 89.22%

New best model saved with Val Accuracy: 52.27%
Epoch 6/100 | Train Loss: 0.8275 | Train Acc: 78.41%
Validation Loss: 2.1486 | Validation Acc: 52.27% | Spar

In [64]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 52.53%


In [65]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [66]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 45s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [67]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 27.8730 | Train Acc: 56.07% | Val Loss: 2.2529 | Val Acc: 51.29% | Sparsity: 89.22%
Epoch 2/50 | Train Loss: 19.7225 | Train Acc: 69.14% | Val Loss: 2.1973 | Val Acc: 53.17% | Sparsity: 89.22%
Epoch 3/50 | Train Loss: 16.4280 | Train Acc: 74.95% | Val Loss: 2.1107 | Val Acc: 53.76% | Sparsity: 89.22%
Epoch 4/50 | Train Loss: 14.0353 | Train Acc: 79.15% | Val Loss: 2.1627 | Val Acc: 54.23% | Sparsity: 89.22%
Epoch 5/50 | Train Loss: 12.1198 | Train Acc: 82.50% | Val Loss: 2.1879 | Val Acc: 53.88% | Sparsity: 89.22%
Epoch 6/50 | Train Loss: 10.4491 | Train Acc: 85.61% | Val Loss: 2.2494 | Val Acc: 52.57% | Sparsity: 89.22%
Epoch 7/50 | Train Loss: 9.0884 | Train Acc: 88.21% | Val Loss: 2.2734 | Val Acc: 52.82% | Sparsity: 89.22%
Epoch 8/50 | Train Loss: 7.8299 | Train Acc: 90.48% | Val Loss: 2.2770 | Val Acc: 52.48% | Sparsity: 89.22%
Epoch 9/50 | Train Loss: 6.7699 | Train Acc: 92.34% | Val Loss: 2.3172 | Val Acc: 52.20% | Sparsity: 89.22%
Early stopping trigger

In [68]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 52.19%


In [69]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [70]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=6.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [71]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy: 0.53%


In [72]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 10.6875 | Train Acc: 56.91% | Val Loss: 2.2260 | Val Acc: 52.06% | Sparsity: 89.22%
Epoch 2/50 | Train Loss: 6.7451 | Train Acc: 73.45% | Val Loss: 2.1636 | Val Acc: 53.09% | Sparsity: 89.22%
Epoch 3/50 | Train Loss: 5.3013 | Train Acc: 79.98% | Val Loss: 2.1820 | Val Acc: 53.86% | Sparsity: 89.22%
Epoch 4/50 | Train Loss: 4.3263 | Train Acc: 84.48% | Val Loss: 2.1997 | Val Acc: 53.38% | Sparsity: 89.22%
Epoch 5/50 | Train Loss: 3.6481 | Train Acc: 87.76% | Val Loss: 2.2236 | Val Acc: 52.86% | Sparsity: 89.22%
Epoch 6/50 | Train Loss: 3.0661 | Train Acc: 90.29% | Val Loss: 2.2727 | Val Acc: 52.94% | Sparsity: 89.22%
Epoch 7/50 | Train Loss: 2.6082 | Train Acc: 92.57% | Val Loss: 2.2733 | Val Acc: 52.87% | Sparsity: 89.22%
Epoch 8/50 | Train Loss: 2.2409 | Train Acc: 94.13% | Val Loss: 2.3140 | Val Acc: 52.44% | Sparsity: 89.22%
Early stopping triggered at epoch 8. No improvement for 5 epochs.
Student model saved before pruning at: pruned_student_retrain_KD_90%.

In [73]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 52.26%


# 80% Sparsity

In [74]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [75]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [76]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 1.08%


In [77]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 50.89%
Epoch 1/100 | Train Loss: 1.1842 | Train Acc: 73.18%
Validation Loss: 2.4256 | Validation Acc: 50.89% | Sparsity: 79.31%

New best model saved with Val Accuracy: 52.31%
Epoch 2/100 | Train Loss: 0.5174 | Train Acc: 87.06%
Validation Loss: 2.3679 | Validation Acc: 52.31% | Sparsity: 79.31%

New best model saved with Val Accuracy: 52.98%
Epoch 3/100 | Train Loss: 0.3434 | Train Acc: 91.36%
Validation Loss: 2.3985 | Validation Acc: 52.98% | Sparsity: 79.31%

New best model saved with Val Accuracy: 53.05%
Epoch 4/100 | Train Loss: 0.2385 | Train Acc: 94.02%
Validation Loss: 2.4798 | Validation Acc: 53.05% | Sparsity: 79.31%

New best model saved with Val Accuracy: 53.98%
Epoch 5/100 | Train Loss: 0.1666 | Train Acc: 96.00%
Validation Loss: 2.5519 | Validation Acc: 53.98% | Sparsity: 79.31%

Epoch 6/100 | Train Loss: 0.1185 | Train Acc: 97.25%
Validation Loss: 2.6098 | Validation Acc: 53.14% | Sparsity: 79.31%

Epoch 7/100 | Train Loss: 0.0847 

In [78]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 53.41%


In [79]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [80]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [81]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 44s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [82]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 19.0211 | Train Acc: 71.34% | Val Loss: 2.6777 | Val Acc: 51.57% | Sparsity: 79.31%
Epoch 2/50 | Train Loss: 12.8240 | Train Acc: 80.88% | Val Loss: 2.6079 | Val Acc: 51.81% | Sparsity: 79.31%
Epoch 3/50 | Train Loss: 9.8287 | Train Acc: 86.15% | Val Loss: 2.3964 | Val Acc: 53.67% | Sparsity: 79.31%
Epoch 4/50 | Train Loss: 7.6236 | Train Acc: 90.11% | Val Loss: 2.5063 | Val Acc: 52.46% | Sparsity: 79.31%
Epoch 5/50 | Train Loss: 5.7970 | Train Acc: 93.44% | Val Loss: 2.3623 | Val Acc: 52.76% | Sparsity: 79.31%
Epoch 6/50 | Train Loss: 4.3215 | Train Acc: 96.04% | Val Loss: 2.3580 | Val Acc: 53.19% | Sparsity: 79.31%
Epoch 7/50 | Train Loss: 3.2049 | Train Acc: 97.87% | Val Loss: 2.3115 | Val Acc: 54.03% | Sparsity: 79.31%
Epoch 8/50 | Train Loss: 2.4023 | Train Acc: 98.97% | Val Loss: 2.2415 | Val Acc: 54.21% | Sparsity: 79.31%
Epoch 9/50 | Train Loss: 1.8065 | Train Acc: 99.58% | Val Loss: 2.1415 | Val Acc: 55.56% | Sparsity: 79.31%
Epoch 10/50 | Train Loss: 

In [83]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 57.95%


In [84]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [85]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 41s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [86]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 4.8227 | Train Acc: 83.67% | Val Loss: 2.2431 | Val Acc: 53.88% | Sparsity: 79.31%
Epoch 2/50 | Train Loss: 2.0896 | Train Acc: 93.96% | Val Loss: 2.2816 | Val Acc: 54.18% | Sparsity: 79.31%
Epoch 3/50 | Train Loss: 1.2789 | Train Acc: 97.50% | Val Loss: 2.2033 | Val Acc: 55.30% | Sparsity: 79.31%
Epoch 4/50 | Train Loss: 0.8890 | Train Acc: 98.93% | Val Loss: 2.2408 | Val Acc: 55.04% | Sparsity: 79.31%
Epoch 5/50 | Train Loss: 0.6656 | Train Acc: 99.60% | Val Loss: 2.2065 | Val Acc: 55.78% | Sparsity: 79.31%
Epoch 6/50 | Train Loss: 0.5401 | Train Acc: 99.84% | Val Loss: 2.1417 | Val Acc: 56.23% | Sparsity: 79.31%
Epoch 7/50 | Train Loss: 0.4719 | Train Acc: 99.92% | Val Loss: 2.1386 | Val Acc: 56.26% | Sparsity: 79.31%
Epoch 8/50 | Train Loss: 0.4268 | Train Acc: 99.94% | Val Loss: 2.1414 | Val Acc: 56.28% | Sparsity: 79.31%
Epoch 9/50 | Train Loss: 0.3950 | Train Acc: 99.96% | Val Loss: 2.1254 | Val Acc: 56.20% | Sparsity: 79.31%
Epoch 10/50 | Train Loss: 0.

In [87]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 56.54%


# 75% of Sparsity

In [88]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [89]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 45s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [90]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 2.63%


In [91]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 52.35%
Epoch 1/100 | Train Loss: 0.7020 | Train Acc: 83.90%
Validation Loss: 2.4453 | Validation Acc: 52.35% | Sparsity: 74.35%

New best model saved with Val Accuracy: 54.02%
Epoch 2/100 | Train Loss: 0.2546 | Train Acc: 93.72%
Validation Loss: 2.4056 | Validation Acc: 54.02% | Sparsity: 74.35%

Epoch 3/100 | Train Loss: 0.1488 | Train Acc: 96.48%
Validation Loss: 2.4950 | Validation Acc: 53.80% | Sparsity: 74.35%

New best model saved with Val Accuracy: 54.21%
Epoch 4/100 | Train Loss: 0.0903 | Train Acc: 98.06%
Validation Loss: 2.5649 | Validation Acc: 54.21% | Sparsity: 74.35%

Epoch 5/100 | Train Loss: 0.0597 | Train Acc: 98.79%
Validation Loss: 2.6377 | Validation Acc: 54.11% | Sparsity: 74.35%

New best model saved with Val Accuracy: 54.27%
Epoch 6/100 | Train Loss: 0.0415 | Train Acc: 99.28%
Validation Loss: 2.6783 | Validation Acc: 54.27% | Sparsity: 74.35%

Epoch 7/100 | Train Loss: 0.0299 | Train Acc: 99.52%
Validation Loss: 2.7515 | V

In [92]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 54.39%


In [93]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

# Retrain with KD

In [94]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [95]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 45s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [96]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 16.3893 | Train Acc: 75.71% | Val Loss: 2.7591 | Val Acc: 50.23% | Sparsity: 74.35%
Epoch 2/50 | Train Loss: 11.1141 | Train Acc: 83.66% | Val Loss: 2.4510 | Val Acc: 51.89% | Sparsity: 74.35%
Epoch 3/50 | Train Loss: 8.1134 | Train Acc: 89.09% | Val Loss: 2.5687 | Val Acc: 52.45% | Sparsity: 74.35%
Epoch 4/50 | Train Loss: 6.0443 | Train Acc: 92.87% | Val Loss: 2.3957 | Val Acc: 53.21% | Sparsity: 74.35%
Epoch 5/50 | Train Loss: 4.2513 | Train Acc: 96.06% | Val Loss: 2.3610 | Val Acc: 52.90% | Sparsity: 74.35%
Epoch 6/50 | Train Loss: 3.0376 | Train Acc: 98.02% | Val Loss: 2.3233 | Val Acc: 53.96% | Sparsity: 74.35%
Epoch 7/50 | Train Loss: 2.1287 | Train Acc: 99.22% | Val Loss: 2.1679 | Val Acc: 55.53% | Sparsity: 74.35%
Epoch 8/50 | Train Loss: 1.5716 | Train Acc: 99.73% | Val Loss: 2.1225 | Val Acc: 56.49% | Sparsity: 74.35%
Epoch 9/50 | Train Loss: 1.2264 | Train Acc: 99.91% | Val Loss: 2.0639 | Val Acc: 57.38% | Sparsity: 74.35%
Epoch 10/50 | Train Loss: 

In [97]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 58.58%


In [98]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

In [99]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [100]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 45s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [101]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 3.0841 | Train Acc: 91.23% | Val Loss: 2.2603 | Val Acc: 54.56% | Sparsity: 74.35%
Epoch 2/50 | Train Loss: 1.1697 | Train Acc: 97.70% | Val Loss: 2.2637 | Val Acc: 54.63% | Sparsity: 74.35%
Epoch 3/50 | Train Loss: 0.7056 | Train Acc: 99.39% | Val Loss: 2.1838 | Val Acc: 55.29% | Sparsity: 74.35%
Epoch 4/50 | Train Loss: 0.5166 | Train Acc: 99.83% | Val Loss: 2.1493 | Val Acc: 56.03% | Sparsity: 74.35%
Epoch 5/50 | Train Loss: 0.4266 | Train Acc: 99.93% | Val Loss: 2.1156 | Val Acc: 56.55% | Sparsity: 74.35%
Epoch 6/50 | Train Loss: 0.3834 | Train Acc: 99.96% | Val Loss: 2.0867 | Val Acc: 56.75% | Sparsity: 74.35%
Epoch 7/50 | Train Loss: 0.3470 | Train Acc: 99.97% | Val Loss: 2.0840 | Val Acc: 56.72% | Sparsity: 74.35%
Epoch 8/50 | Train Loss: 0.3227 | Train Acc: 99.98% | Val Loss: 2.0773 | Val Acc: 57.03% | Sparsity: 74.35%
Epoch 9/50 | Train Loss: 0.3055 | Train Acc: 99.98% | Val Loss: 2.0824 | Val Acc: 56.68% | Sparsity: 74.35%
Epoch 10/50 | Train Loss: 0.

In [102]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 57.42%


## 50% Sparsity

In [103]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [104]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 43s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [105]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 27.79%


In [106]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 57.59%
Epoch 1/200 | Train Loss: 0.0226 | Train Acc: 99.65%
Validation Loss: 2.2924 | Validation Acc: 57.59% | Sparsity: 50.02%

New best model saved with Val Accuracy: 57.82%
Epoch 2/200 | Train Loss: 0.0086 | Train Acc: 99.89%
Validation Loss: 2.2752 | Validation Acc: 57.82% | Sparsity: 50.02%

Epoch 3/200 | Train Loss: 0.0059 | Train Acc: 99.94%
Validation Loss: 2.2870 | Validation Acc: 57.57% | Sparsity: 50.02%

Epoch 4/200 | Train Loss: 0.0045 | Train Acc: 99.97%
Validation Loss: 2.3075 | Validation Acc: 57.57% | Sparsity: 50.02%

Epoch 5/200 | Train Loss: 0.0041 | Train Acc: 99.96%
Validation Loss: 2.3115 | Validation Acc: 57.67% | Sparsity: 50.02%

New best model saved with Val Accuracy: 57.89%
Epoch 6/200 | Train Loss: 0.0032 | Train Acc: 99.97%
Validation Loss: 2.3337 | Validation Acc: 57.89% | Sparsity: 50.02%

New best model saved with Val Accuracy: 57.98%
Epoch 7/200 | Train Loss: 0.0031 | Train Acc: 99.97%
Validation Loss: 2.3001 | V

In [107]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 58.53%


In [108]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [109]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 44s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [110]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 7.6188 | Train Acc: 90.40% | Val Loss: 2.6825 | Val Acc: 50.49% | Sparsity: 50.02%
Epoch 2/50 | Train Loss: 8.4020 | Train Acc: 88.52% | Val Loss: 2.6937 | Val Acc: 51.04% | Sparsity: 50.02%
Epoch 3/50 | Train Loss: 5.2672 | Train Acc: 94.08% | Val Loss: 2.5011 | Val Acc: 53.16% | Sparsity: 50.02%
Epoch 4/50 | Train Loss: 3.2214 | Train Acc: 97.60% | Val Loss: 2.3296 | Val Acc: 55.88% | Sparsity: 50.02%
Epoch 5/50 | Train Loss: 1.9449 | Train Acc: 99.33% | Val Loss: 2.1442 | Val Acc: 57.25% | Sparsity: 50.02%
Epoch 6/50 | Train Loss: 1.2713 | Train Acc: 99.86% | Val Loss: 2.0737 | Val Acc: 58.65% | Sparsity: 50.02%
Epoch 7/50 | Train Loss: 0.9714 | Train Acc: 99.96% | Val Loss: 2.0289 | Val Acc: 59.13% | Sparsity: 50.02%
Epoch 8/50 | Train Loss: 0.8093 | Train Acc: 99.97% | Val Loss: 1.9882 | Val Acc: 59.89% | Sparsity: 50.02%
Epoch 9/50 | Train Loss: 0.7132 | Train Acc: 99.98% | Val Loss: 1.9695 | Val Acc: 60.02% | Sparsity: 50.02%
Epoch 10/50 | Train Loss: 0.

In [111]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 60.93%


In [112]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [113]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 4m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.4386 | Train Acc: 99.92% | Val Loss: 2.1028 | Val Acc: 58.50% | Sparsity: 50.02%
Epoch 2/50 | Train Loss: 0.2524 | Train Acc: 99.98% | Val Loss: 2.0408 | Val Acc: 59.21% | Sparsity: 50.02%
Epoch 3/50 | Train Loss: 0.2250 | Train Acc: 99.98% | Val Loss: 2.0560 | Val Acc: 58.99% | Sparsity: 50.02%
Epoch 4/50 | Train Loss: 0.2084 | Train Acc: 99.98% | Val Loss: 2.0333 | Val Acc: 59.06% | Sparsity: 50.02%
Epoch 5/50 | Train Loss: 0.1974 | Train Acc: 99.98% | Val Loss: 2.0252 | Val Acc: 59.21% | Sparsity: 50.02%
Epoch 6/50 | Train Loss: 0.1884 | Train Acc: 99.98% | Val Loss: 2.0225 | Val Acc: 59.21% | Sparsity: 50.02%
Epoch 7/50 | Train Loss: 0.1828 | Train Acc: 99.98% | Val Loss: 2.0204 | Val Acc: 59.39% | Sparsity: 50.02%
Epoch 8/50 | Train Loss: 0.1775 | Train Acc: 99.98% | Val Loss: 2.0089 | Val Acc: 59.38% | Sparsity: 50.02%
Epoch 9/50 | Train Loss: 0.1736 | Train Acc: 99.98% | Val Loss: 2.0135 | Val Acc: 59.47% | Sparsity: 50.02%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")